# A real use case

![ngs sequencing](http://www.genohm.com/wp-content/uploads/2015/05/DNA-actg1.jpg)

## Preparation

In [52]:
# Base dir
mydir = './tmp/'
# Python variables
myfile = mydir + 'job.py'
myinput = mydir + 'ngs.sam'
myshortinput = myinput + '.short'
ngs_module = mydir + 'ngs.py'
# Bash variables
%env mydir $mydir
%env myinput $myinput
%env myshortinput $myshortinput

env: mydir=./tmp/
env: myinput=./tmp/ngs.sam
env: myshortinput=./tmp/ngs.sam.short


In [53]:
%%bash
mkdir -p $mydir
head -1500 $myinput > $myshortinput
wc -l $mydir*sam*

   334121 ./tmp/ngs.sam
     1500 ./tmp/ngs.sam.short
   335621 total


### Get lines

In [54]:
file="/data/lectures/ttmda/hadoop/tmp/ngs.sam"

In [55]:
out = ! head -150 $file | tail -n 1
mline = out.pop()

out = ! head -1400 $file | tail -n 1
oline = out.pop()

print(mline, oline)

HSCAN:421:C47DAACXX:4:1208:18817:185709	161	chrM	802	37	67M1D8M	=	797	69	CACCCCCACGGGAAACAGCAGTGATTAACCTTTAGCAATAAACGAAAGTTTAACTAAGCTATACTAACCCTGTCT	CCCFFFFFHHHGHJJIJGIJJJIGIJJJJIIJJJJJJEIJJIJGI=FHGIHHGGIIGIHBHFGHFFFFFDECCCC	X0:i:1	X1:i:0	MD:Z:67^C3A1G0G1	RG:Z:1	XG:i:1	AM:i:37	NM:i:4	SM:i:37	XM:i:3	XO:i:1	MQ:i:37	XT:A:U HSCAN:421:C47DAACXX:2:1207:3127:60980	121	chr1	142638010	18	5M1I69M	=	142638010	0	AGACAGGCTTGTCCTCTGGCCACAAACTATTAACGTTTCTCCCACATGGAAATTATGCTTTGCCCCTCTCAAGAG	>;@DCBB@;7=;HEIHE@F@C@IGGGIGFBGJIIHDG?B?AGHDGFEIGGJJJIIGIIIHGIHDFHFFDFFFC@@	X0:i:1	X1:i:3	XA:Z:chr1,-142638008,1M2D4M1I69M,4;chrUn_gl000221,+60004,68M1I6M,4;chr21,-10183846,5M1I69M,4;	MD:Z:0C0C72	RG:Z:1	XG:i:1	AM:i:0	NM:i:3	SM:i:18	XM:i:2	XO:i:1	XT:A:U


## Constants

In [56]:
# Symbol for header inside SAM format
HEADER_CHAR = "@"
# Chromosomic separator
SEP = ":"
# Formats for quality reading: http://j.mp/1zo30bt
# Illumina 1.8+ Phred+33,  raw reads typically (0, 41)
PHRED_INIT = 33
# What is the minimum quality?
PHRED_THOLD = 20  
# Handle STRAND:
FORWARD_STRAND_FLAG = 16

Writing to a separate module

In [57]:
%%writefile $ngs_module

""" NGS functions """

# Symbol for header inside SAM format
HEADER_CHAR = "@"
# Chromosomic separator
SEP = ":"
# Formats for quality reading: http://j.mp/1zo30bt
# Illumina 1.8+ Phred+33,  raw reads typically (0, 41)
PHRED_INIT = 33
# What is the minimum quality?
PHRED_THOLD = 20  
# Handle STRAND:
FORWARD_STRAND_FLAG = 16



Overwriting ./tmp/ngs.py


Handling the strand

In [58]:
def compute_strand(flag, myseq='ACTG', mystart=1):
    # Convert the flag to decimal, and check
    # the bit in the 5th position from the right.
    if flag & FORWARD_STRAND_FLAG:       # Strand forward
        mystop = mystart + len(myseq)
    else:               # Strand reverse
        mystop = mystart + 1
        mystart = mystop - len(myseq)
    return mystart, mystop


def ngs_split(line=None):
    """ Recover necessary data for NGS analysis """
    pieces = line.split("\t")

    myflag = int(pieces[1])
    mychr = pieces[2]
    mystart = int(pieces[3])
    myseq = pieces[9]
    myqc = pieces[10]
    mystart, mystop = compute_strand(myflag, myseq, mystart)

    shortchr = mychr[3:]
    if shortchr.isdigit():
        code = shortchr.zfill(2)  # sortable chromosome code
    else:
        code = ord(shortchr).__str__()

    return mychr, mystart, mystop, myseq, myflag, myqc, code


In [59]:
ngs_split(mline)

('chrM',
 728,
 803,
 'CACCCCCACGGGAAACAGCAGTGATTAACCTTTAGCAATAAACGAAAGTTTAACTAAGCTATACTAACCCTGTCT',
 161,
 'CCCFFFFFHHHGHJJIJGIJJJIGIJJJJIIJJJJJJEIJJIJGI=FHGIHHGGIIGIHBHFGHFFFFFDECCCC',
 '77')

In [60]:
ngs_split(oline)

('chr1',
 142638010,
 142638085,
 'AGACAGGCTTGTCCTCTGGCCACAAACTATTAACGTTTCTCCCACATGGAAATTATGCTTTGCCCCTCTCAAGAG',
 121,
 '>;@DCBB@;7=;HEIHE@F@C@IGGGIGFBGJIIHDG?B?AGHDGFEIGGJJJIIGIIIHGIHDFHFFDFFFC@@',
 '01')

Append tested functions into ngs module

In [62]:
%%writefile -a $ngs_module

def compute_strand(flag, myseq='ACTG', mystart=1):
    # Convert the flag to decimal, and check
    # the bit in the 5th position from the right.
    if flag & FORWARD_STRAND_FLAG:       # Strand forward
        mystop = mystart + len(myseq)
    else:               # Strand reverse
        mystop = mystart + 1
        mystart = mystop - len(myseq)
    return mystart, mystop



Appending to ./tmp/ngs.py


In [63]:
%%writefile -a $ngs_module

def ngs_split(line=None):
    """ Recover necessary data for NGS analysis """
    pieces = line.split("\t")

    myflag = int(pieces[1])
    mychr = pieces[2]
    mystart = int(pieces[3])
    myseq = pieces[9]
    myqc = pieces[10]
    mystart, mystop = compute_strand(myflag, myseq, mystart)

    shortchr = mychr[3:]
    if shortchr.isdigit():
        code = shortchr.zfill(2)  # sortable chromosome code
    else:
        code = ord(shortchr).__str__()

    return mychr, mystart, mystop, myseq, myflag, myqc, code



Appending to ./tmp/ngs.py


We can check with nb editor the final result

In [79]:
# recover data needed from one line
mychr, mystart, mystop, myseq, _, myqc, _ = ngs_split(oline)

# Cycle
for i in range(mystart, mystop):
    mypos = i - mystart
    current_qc = ord(myqc[mypos]) - PHRED_INIT  # quality value
    if current_qc > PHRED_THOLD:
        label = mychr + SEP + i.__str__()
        #label = code + SEP + i.__str__() + SEP + mychr
        current_letter = myseq[mypos]
        print(label, current_letter)

chr1:142638010 A
chr1:142638011 G
chr1:142638012 A
chr1:142638013 C
chr1:142638014 A
chr1:142638015 G
chr1:142638016 G
chr1:142638017 C
chr1:142638018 T
chr1:142638019 T
chr1:142638020 G
chr1:142638021 T
chr1:142638022 C
chr1:142638023 C
chr1:142638024 T
chr1:142638025 C
chr1:142638026 T
chr1:142638027 G
chr1:142638028 G
chr1:142638029 C
chr1:142638030 C
chr1:142638031 A
chr1:142638032 C
chr1:142638033 A
chr1:142638034 A
chr1:142638035 A
chr1:142638036 C
chr1:142638037 T
chr1:142638038 A
chr1:142638039 T
chr1:142638040 T
chr1:142638041 A
chr1:142638042 A
chr1:142638043 C
chr1:142638044 G
chr1:142638045 T
chr1:142638046 T
chr1:142638047 T
chr1:142638048 C
chr1:142638049 T
chr1:142638050 C
chr1:142638051 C
chr1:142638052 C
chr1:142638053 A
chr1:142638054 C
chr1:142638055 A
chr1:142638056 T
chr1:142638057 G
chr1:142638058 G
chr1:142638059 A
chr1:142638060 A
chr1:142638061 A
chr1:142638062 T
chr1:142638063 T
chr1:142638064 A
chr1:142638065 T
chr1:142638066 G
chr1:142638067 C
chr1:142638068

In [64]:
%%writefile $myfile

from mrjob.job import MRJob
try:
    from ngs import ngs_split, PHRED_INIT, PHRED_THOLD, HEADER_CHAR, SEP
except:
    pass

class MRcoverage(MRJob):
    """ Map Reduce for NGS coverage computation"""

    def mapper(self, _, line):
        """ Split data into keys and value """

        if line[0] == HEADER_CHAR:
            yield "header", 1
        else:
            # Get data
            mychr, mystart, mystop, myseq, myflag, myqc, code = ngs_split(line)
            # For each base of my dna sequence
            for i in range(mystart, mystop):
                mypos = i - mystart
                current_qc = ord(myqc[mypos]) - PHRED_INIT  # quality value
                if current_qc > PHRED_THOLD:
                    # Choose the key to emit
                    label = code + SEP + i.__str__() + SEP + mychr
                    current_letter = myseq[mypos]
                    yield label, 1
                    yield label + SEP + current_letter, 1

    def reducer(self, key, values):
        """ Sum up values"""
        yield key, sum(values)

if __name__ == '__main__':
    MRcoverage().run()


Overwriting ./tmp/job.py


In [65]:
! python $myfile $myshortinput

using configs in /etc/mrjob.conf
creating tmp directory /tmp/job.jovyan.20151214.223339.026717
writing to /tmp/job.jovyan.20151214.223339.026717/step-0-mapper_part-00000
Counters from step 1:
  (none found)
writing to /tmp/job.jovyan.20151214.223339.026717/step-0-mapper-sorted
> sort /tmp/job.jovyan.20151214.223339.026717/step-0-mapper_part-00000
writing to /tmp/job.jovyan.20151214.223339.026717/step-0-reducer_part-00000
Counters from step 1:
  (none found)
Moving /tmp/job.jovyan.20151214.223339.026717/step-0-reducer_part-00000 -> /tmp/job.jovyan.20151214.223339.026717/output/part-00000
Streaming final output from /tmp/job.jovyan.20151214.223339.026717/output
"01:10000:chr1"	2
"01:10000:chr1:A"	2
"01:10001:chr1"	2
"01:10001:chr1:C"	2
"01:10002:chr1"	2
"01:10002:chr1:A"	2
"01:10003:chr1"	2
"01:10003:chr1:G"	2
"01:10004:chr1"	2
"01:10004:chr1:C"	2
"01:10005:chr1"	2
"01:10005:chr1:C"	2
"01:10006:chr1"	2
"01:10006:chr1:C"	2
"01:10007:chr1"	2
"01:10007:chr1:T"	2
"01:10008:chr1"	2
"01:10008:

## This is not going to work

In [66]:
! python $myfile -r hadoop $myshortinput 1> log.out 2> log.err

In [67]:
cat log.err

using configs in /etc/mrjob.conf
Using Hadoop version 2.6.0
Copying local files into hdfs:///user/jovyan/tmp/mrjob/job.jovyan.20151214.223411.865154/files/
HADOOP: packageJobJar: [/tmp/hadoop-unjar1415851052693945384/] [] /tmp/streamjob2494797705479571749.jar tmpDir=null
HADOOP: Connecting to ResourceManager at /0.0.0.0:8032
HADOOP: Connecting to ResourceManager at /0.0.0.0:8032
HADOOP: Total input paths to process : 1
HADOOP: number of splits:2
HADOOP: Submitting tokens for job: job_1450124645745_0008
HADOOP: Submitted application application_1450124645745_0008
HADOOP: The url to track the job: http://sparker:8088/proxy/application_1450124645745_0008/
HADOOP: Running job: job_1450124645745_0008
HADOOP: Job job_1450124645745_0008 running in uber mode : false
HADOOP:  map 0% reduce 0%
HADOOP: Task Id : attempt_1450124645745_0008_m_000001_0, Status : FAILED
HADOOP: Error: java.lang.RuntimeException: PipeMapRed.waitOutputThreads(): subprocess failed with code 1
HADOOP: 	at org.apache.hado

Hint: your modules could be compressed and sent to HADOOP or EMR

http://mrjob.readthedocs.org/en/latest/guides/setup-cookbook.html#putting-your-source-tree-in-pythonpath

In [68]:
cat $ngs_module $myfile > tmp/hadoop.py

In [69]:
%reload_ext mrjobmagic

In [70]:
%mapreduce $myshortinput tmp/hadoop.py hadoop

Input file is: ./tmp/ngs.sam.short
File provided by user
Executing Mrjob.
Done!

Files:
tmp/hadoop.py  tmp/hadoop.py.err  tmp/hadoop.py.out


{'77:12041:chrM:A': 10,
 '01:1844944:chr1:T': 1,
 '77:6018:chrM': 1,
 '01:52553146:chr1:G': 13,
 '01:142592605:chr1:C': 1,
 '01:142558427:chr1': 1,
 '77:7868:chrM:A': 2,
 '01:142566138:chr1': 1,
 '77:2301:chrM:A': 3,
 '01:142587677:chr1': 2,
 '77:14348:chrM': 11,
 '01:142640707:chr1': 1,
 '77:8269:chrM:G': 2,
 '01:110654257:chr1': 1,
 '77:7094:chrM': 9,
 '77:8789:chrM': 2,
 '01:142619284:chr1': 3,
 '01:142688079:chr1': 1,
 '01:10410353:chr1:C': 1,
 '01:234571:chr1:G': 1,
 '77:10523:chrM': 8,
 '01:94972619:chr1': 1,
 '77:11212:chrM:T': 3,
 '01:142553860:chr1:C': 1,
 '01:142619348:chr1:G': 1,
 '77:15400:chrM:A': 1,
 '77:8235:chrM': 5,
 '01:142594964:chr1:A': 1,
 '77:12085:chrM': 7,
 '01:142625524:chr1': 1,
 '01:4246837:chr1:T': 1,
 '01:52553693:chr1:T': 5,
 '77:9498:chrM': 2,
 '77:15054:chrM': 3,
 '01:117521001:chr1:C': 1,
 '01:142619700:chr1:T': 2,
 '77:14425:chrM:T': 3,
 '01:142616850:chr1:T': 1,
 '77:5606:chrM': 1,
 '77:10916:chrM': 5,
 '77:4965:chrM:C': 6,
 '01:142594912:chr1': 1,
 '

In [71]:
data = _

In [72]:
data

{'77:12041:chrM:A': 10,
 '01:1844944:chr1:T': 1,
 '77:6018:chrM': 1,
 '01:52553146:chr1:G': 13,
 '01:142592605:chr1:C': 1,
 '01:142558427:chr1': 1,
 '77:7868:chrM:A': 2,
 '01:142566138:chr1': 1,
 '77:2301:chrM:A': 3,
 '01:142587677:chr1': 2,
 '77:14348:chrM': 11,
 '01:142640707:chr1': 1,
 '77:8269:chrM:G': 2,
 '01:110654257:chr1': 1,
 '77:7094:chrM': 9,
 '77:8789:chrM': 2,
 '01:142619284:chr1': 3,
 '01:142688079:chr1': 1,
 '01:10410353:chr1:C': 1,
 '01:234571:chr1:G': 1,
 '77:10523:chrM': 8,
 '01:94972619:chr1': 1,
 '77:11212:chrM:T': 3,
 '01:142553860:chr1:C': 1,
 '01:142619348:chr1:G': 1,
 '77:15400:chrM:A': 1,
 '77:8235:chrM': 5,
 '01:142594964:chr1:A': 1,
 '77:12085:chrM': 7,
 '01:142625524:chr1': 1,
 '01:4246837:chr1:T': 1,
 '01:52553693:chr1:T': 5,
 '77:9498:chrM': 2,
 '77:15054:chrM': 3,
 '01:117521001:chr1:C': 1,
 '01:142619700:chr1:T': 2,
 '77:14425:chrM:T': 3,
 '01:142616850:chr1:T': 1,
 '77:5606:chrM': 1,
 '77:10916:chrM': 5,
 '77:4965:chrM:C': 6,
 '01:142594912:chr1': 1,
 '

## Counters

In [89]:
%%writefile $myfile

from mrjob.job import MRJob
from ngs import ngs_split, PHRED_INIT, PHRED_THOLD, HEADER_CHAR, SEP

class MRcoverage(MRJob):

    def mapper(self, _, line):
        if line[0] != HEADER_CHAR:
            mychr, mystart, mystop, myseq, myflag, myqc, code = ngs_split(line)
            # Save chromosome info
            self.increment_counter('chromosomes', mychr, 1)

            for i in range(mystart, mystop):
                mypos = i - mystart
                current_qc = ord(myqc[mypos]) - PHRED_INIT  # quality value
                if current_qc > PHRED_THOLD:
                    yield mychr + SEP + str(i) + SEP + myseq[mypos], 1

    def reducer(self, key, values):
        """ Sum up values"""
        yield key, sum(values)

if __name__ == '__main__':
    MRcoverage().run()


Overwriting ./tmp/job.py


In [90]:
! python $myfile $myshortinput 1> log.out 2> log.err

In [91]:
cat log.err

using configs in /etc/mrjob.conf
creating tmp directory /tmp/job.jovyan.20151214.225103.618125
writing to /tmp/job.jovyan.20151214.225103.618125/step-0-mapper_part-00000
Counters from step 1:
	chromosomes
		chr1=500
		chrM=903
writing to /tmp/job.jovyan.20151214.225103.618125/step-0-mapper-sorted
> sort /tmp/job.jovyan.20151214.225103.618125/step-0-mapper_part-00000
writing to /tmp/job.jovyan.20151214.225103.618125/step-0-reducer_part-00000
Counters from step 1:
	chromosomes
		chr1=500
		chrM=903
Moving /tmp/job.jovyan.20151214.225103.618125/step-0-reducer_part-00000 -> /tmp/job.jovyan.20151214.225103.618125/output/part-00000
Streaming final output from /tmp/job.jovyan.20151214.225103.618125/output
removing tmp directory /tmp/job.jovyan.20151214.225103.618125
